In [45]:
%matplotlib ipympl
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.signal import welch, periodogram
import os

import matplotlib.dates as mdates

import utide

from physoce import tseries as ts

from ADCP import rditext_to_dataset

### Load data

#### Rover

In [46]:
# Load data
rover_csv_file = 'data/Rover_II_Current_Mag_Hourly_Avg_pad_2018.csv'
df_all = pd.read_csv(rover_csv_file,parse_dates=[7])

# Datetime index
df_all = df_all.set_index('Date_time_R')

# Create u and v components with units m/s
df_all['u'] = df_all['Easting (cm/sec)']/100
df_all['v'] = df_all['Northing (cm/sec)']/100

# mask erroneous points 
# abrupt shift near end of deployment 
bi = ((df_all.index > np.datetime64('2015-06-18T12:00')) &
    (df_all.index < np.datetime64('2015-06-20T00:00')))

df_all['u'][bi] = np.nan
df_all['v'][bi] = np.nan

# select time range
t1 = np.datetime64('2014-10-11T00:00:00')
t2 = np.datetime64('2018-10-17T00:00:00')
df_all = df_all.loc[(df_all.index >= t1) & (df_all.index <= t2)]

# tidal analysis
time = mdates.date2num(df_all.index)

coef = utide.solve(time, np.array(df_all['u']), 
         np.array(df_all['v']),
         lat=35+8.4585/60,
         method='robust')

crit = (coef['diagn']['SNR'] > 2) & (1/coef['aux']['frq'] < 33)

# resample and interpolate
df_all_r = df_all.resample('1H').mean()

# Calculate tidal currents at hourly points
time_r = mdates.date2num(df_all_r.index)
#tide_r = utide.reconstruct(time_r,coef)
tide_r = utide.reconstruct(time_r,coef,constit=coef['name'][crit])
df_all_r['u_tide'] = tide_r.u
df_all_r['v_tide'] = tide_r.v
df_all_r['u_detide'] = df_all_r['u'] - df_all_r['u_tide']
df_all_r['v_detide'] = df_all_r['v'] - df_all_r['v_tide']

df_all_int = df_all_r.interpolate(limit=int(24*3.5))

df_all_int['u'] = df_all_int['u_detide'] + df_all_int['u_tide']
df_all_int['v'] = df_all_int['v_detide'] + df_all_int['v_tide']

df_all_int['u_f'] = ts.pl64(df_all_int['u_detide'])
df_all_int['v_f'] = ts.pl64(df_all_int['v_detide'])

df_all_int['u_super'] = df_all_int['u'] - df_all_int['u_f'] - df_all_int['u_tide']
df_all_int['v_super'] = df_all_int['v'] - df_all_int['v_f'] - df_all_int['v_tide']

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


solve: matrix prep ... solution ... diagnostics ... done.
prep/calcs ... done.


/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:251: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()


#### ADCP

In [47]:
ds = xr.open_dataset('data/MBARI_StationM_ADCP_201711_201811.nc')

### Resample to hourly values

#### Define date range

In [48]:
t1 = np.datetime64('2017-11-12T00:00:00')
t2 = np.datetime64('2018-10-17T00:00:00')

#### ADCP

In [49]:
dsr = ds.resample({'time':'1H'}).mean()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/xarray/core/nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [60]:
ti, = np.where((dsr['time'] >= t1) & (dsr['time'] < t2))
dsf = dsr.isel(time = ti)

#### Rover

In [61]:
dff = df_all_int.loc[(df_all_int.index >= t1) & (df_all_int.index <= t2)]

## Plot comparisons

In [62]:
plt.figure(figsize=(7.5,4))
plt.plot(dff.index,dff['u'])
plt.plot(dsf['time'],dsf['Eas'][:,1]+0.1,'-')
yl = plt.ylim()

plt.figure(figsize=(7.5,4))
plt.plot(dff.index,dff['v'])
plt.plot(dsf['time'],dsf['Nor'][:,1]+0.1,'-')
yl = plt.ylim()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [63]:
plt.figure(figsize=(7.5,4))
plt.plot(dff.index,ts.pl64(dff['u']))
plt.plot(dsf['time'],ts.pl64(dsf['Eas'][:,1]),'-')
yl = plt.ylim()
plt.legend(['Rover','ADCP'])

plt.figure(figsize=(7.5,4))
plt.plot(dff.index,ts.pl64(dff['v']))
plt.plot(dsf['time'],ts.pl64(dsf['Nor'][:,1]),'-')
yl = plt.ylim()
plt.legend(['Rover','ADCP'])

FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:251: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()


FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:251: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()


In [64]:
plt.figure()
plt.plot(dff['u'],dsf['Eas'][:,1],'.')
plt.xlabel('Rover')
plt.ylabel('ADCP')

plt.figure()
plt.plot(dff['v'],dsf['Nor'][:,1],'.')
plt.xlabel('Rover')
plt.ylabel('ADCP')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Text(0, 0.5, 'ADCP')

In [96]:
gi, = np.where(np.isfinite(dff['v']+dsf['Nor'][:,1]))

resu = stats.linregress(dff['u'][gi],dsf['Eas'][:,1][gi])
resv = stats.linregress(dff['v'][gi],dsf['Nor'][:,1][gi])

print(resu)
print(resv)

plt.figure()
plt.plot(ts.pl64(dff['u'])[::33],ts.pl64(dsf['Eas'][:,1])[::33],'.')
plt.xlabel('Rover u')
plt.ylabel('ADCP u')

plt.figure()
plt.plot(ts.pl64(dff['v'])[::33],ts.pl64(dsf['Nor'][:,1])[::33],'.')
plt.xlabel('Rover v')
plt.ylabel('ADCP v')

plt.figure()
plt.plot(ts.pl64(dsf['Nor'][:,0])[::33],ts.pl64(dsf['Nor'][:,16])[::33],'.')
plt.xlabel('ADCP v - lower bin')
plt.ylabel('ADCP v - upper bin')

LinregressResult(slope=1.1403954006695474, intercept=-0.002560506867169395, rvalue=0.7117105507001336, pvalue=0.0, stderr=0.012480428802283831)
LinregressResult(slope=1.1950035333236817, intercept=-0.0027733025528093795, rvalue=0.7064214195599829, pvalue=0.0, stderr=0.013275735045871278)


/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:251: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the

FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:251: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the

FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:251: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()


Text(0, 0.5, 'ADCP v - upper bin')

In [66]:
dsf['Eas'][:,1][::33]

<xarray.DataArray 'Eas' (time: 247)>
array([0.040333, 0.062167, 0.03775 , ..., 0.006167, 0.012167, 0.007833])
Coordinates:
  * time     (time) datetime64[ns] 2017-11-12 ... 2018-10-16T06:00:00

### Spectral analysis

In [77]:
Eas = np.array(ds['Eas'])
Nor = np.array(ds['Nor'])
Eas_tide = np.array(ds['Eas_tide'])
Nor_tide = np.array(ds['Nor_tide'])

u_rover = dff['u']
v_rover = dff['v']

N = len(Eas)
fE,SE = welch(Eas[:,5],fs=288.,nperseg=N/6,window='hanning')
fN,SN = welch(Nor[:,5],fs=288.,nperseg=N/6,window='hanning')

fur,Sur = welch(u_rover,fs=24.,nperseg=N/6/20,window='hanning')
fvr,Svr = welch(v_rover,fs=24.,nperseg=N/6/20,window='hanning')

#rotary spectrum
fr,Sr = welch(Eas[:,5]+1j*Nor[:,5],
              fs=288.,nperseg=N/6,
              window='hann',return_onesided=False)
icw, = np.where(fr<0)
iccw, = np.where(fr>0)
fcw = -fr[icw]
Scw = Sr[icw]
fccw = fr[iccw]
Sccw = Sr[iccw]

In [92]:
plt.figure()
plt.loglog(fur,Sur+Svr)
plt.loglog(fE,SE+SN,'r-')
plt.legend(['Rover','ADCP'])
plt.ylabel('[m$^2$ s$^{-2}$ cpd$^{-1}$]')
xl = [0.1,30]
plt.xlim(xl)

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

(0.1, 30)

In [68]:
plt.figure()
plt.loglog(fcw,Scw)
plt.loglog(fccw,Sccw,'--')
plt.legend(['clockwise','counter-clockwise'])
plt.xlabel('frequency [cpd]')
plt.ylabel('[m$^2$ s$^{-2}$ cpd$^{-1}$]')
plt.title('rotary spectra [ADCP]')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'rotary spectra [ADCP]')

In [15]:
#periodogram method
fup,Sup = periodogram(Eas[:,5],
              fs=288.,
              window='hanning')

# average frequency bands
navg = 4   # number of frequency bands to average
nfreq = int(np.floor(len(fup)/navg))  # number of freqencies in averaged spectrum

fwt = 1/navg
fm = np.zeros(nfreq)
Sm = np.zeros(nfreq)
for k in range(navg):
    fm = fm + fwt*fup[1:(nfreq*navg)][::navg]
    Sm = Sm + fwt*Sup[1:(nfreq*navg)][::navg]
    
#welch
fw,Sw = welch(Eas[:,5],
              fs=288.,nperseg=N/6,
              window='hann',return_onesided=True)

In [16]:
plt.figure()
plt.loglog(fm,Sm)
plt.loglog(fw,Sw)

FigureCanvasNbAgg()

In [17]:
#rotary spectrum
frp,Srp = periodogram(Eas[:,5]+1j*Nor[:,5],
              fs=288.,
              window='boxcar',return_onesided=False)
icw, = np.where(frp<0)
iccw, = np.where(frp>0)
fcw = -frp[icw]
Scw = Srp[icw]
fccw = frp[iccw]
Sccw = Srp[iccw]

In [18]:
plt.figure()
plt.loglog(fcw,Scw)
plt.loglog(fccw,Sccw,'--')
plt.legend(['clockwise','counter-clockwise'])
plt.xlabel('frequency [cpd]')
plt.ylabel('[m$^2$ s$^{-2}$ cpd$^{-1}$]')
plt.title('rotary spectra - periodogram [ADCP]')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'rotary spectra - periodogram [ADCP]')

In [19]:
u = np.array(dff['u'])
v = np.array(dff['v'])

#rotary spectrum
frp,Srp = periodogram(u+1j*v,
              fs=24.,
              window='boxcar',return_onesided=False,scaling='density')
icw, = np.where(frp<0)
iccw, = np.where(frp>0)
fcw = -frp[icw]
Scw = Srp[icw]
fccw = frp[iccw]
Sccw = Srp[iccw]

In [20]:
plt.figure()
plt.loglog(fcw,Scw)
plt.loglog(fccw,Sccw,'--')
plt.legend(['clockwise','counter-clockwise'])
plt.xlabel('frequency [cpd]')
plt.ylabel('[m$^2$ s$^{-2}$ cpd$^{-1}$]')
plt.title('rotary spectra - periodogram [Rover]')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'rotary spectra - periodogram [Rover]')

In [21]:
dfsub = df.loc[(df.index >= t1) & (df.index <= t2) & 
                (df.index != np.datetime64('NaT'))]

tdsub = np.array(dfsub.index-dfsub.index[0])/np.timedelta64(1,'D')

In [22]:
# DFT

def lombscargle_broadcast(t,x,ofac=4,hifac=1,t0=None,return_onesided=True,return_zero=False):
    '''
    
    Compute the discrete Fourier transform and periodogram for unevenly-spaced 
    data using the Lomb-Scargle periodogram. Follows methods outlined in Scargle
    (1989).
    
    THIS VERSION USES ARRAY BROADCASTING, WHICH TAKES UP TOO MUCH MEMORY
    
    INPUTS
    
    t - array of numerical time values (length N)
    x - array of data values  (length N)
    
    RETURNS
    
    f - array of frequencies
    ftx - array of complex coefficients
          discrete Fourier transform of x
    px - periodogram of ftx, calculated as (1/N)*|ftx|**2
    
    OPTIONAL PARAMETERS
    
    ofac - oversampling parameter 
           ratio of number of frequencies used to number of samples in x
           (default 4)        
    hifac - high frequency parameter
            ratio of highest frequency to pseudo-Nyquist frequency
            (default 1)  
    t0 - time origin
         reference point for phase calculation
         (default - None, first value in t array is used)
    return_onesided - boolean for returning a one-sided spectrum
        If True, return a one-sided spectrum for real data. 
        If False return a two-sided spectrum for real data.  
        Note that for complex data, a two-sided spectrum is always returned.
        (default - True)
    return_zero - boolean for evaluating zero frequency
        If True, include zero frequency. If False, do not include zero frequency.
        Uses expressions for the limit as frequency approaches zero, following
        Scargle (1989). 
        (default - False)
      
    REFERENCE
    
    Scargle, J.D. (1989) Studies in astronomical time series analysis III: Fourier transforms,
    autocorrelation functions, and cross-correlation functions of unevenly spaced data. The
    Astrophysical Journal, 343, 874-887
    '''

    intm = np.mean(np.diff(t))

    flo = ((intm)**-1)/(len(x)*ofac)  # lowest freq
    fhi = hifac*(2*intm)**-1          # highest freq

    f = np.arange(flo,fhi+flo,flo)
    
    if return_zero == True:
        f = np.append(0,f)
        
    # if complex, evaluate two-sided spectrum regardless of user choice
    if np.any(np.iscomplex(x)):
        return_onesided = False
    
    # two-sided spectrum
    if return_onesided == False:
        if return_zero == True:
            f = np.append(-f[1:][::-1],f)
        else:
            f = np.append(-f[::-1],f)

    # time origin (reference point for phase calculation)
    if t0 is None:
        t0 = t[0] 
        
    i = 1j # square root of -1
    N = len(x) # number of samples

    # initialize DFT as array of complex numbers
    ftx = np.nan*np.ones(len(f)) + i*np.nan*np.ones(len(f))

    k = np.nonzero(f)
    k0, = np.where(f == 0)
    
    wrun = 2*np.pi*f[k] # angular frequency    

    Fo = ((N/2)**0.5)*np.exp(-i*wrun*t0) 

    wrun = wrun[:,np.newaxis]
    
    tau = np.arctan2(np.sum(np.sin(2*wrun*t)),np.sum(np.cos(2*wrun*t)))/(2*wrun)
    tprime = t - tau
    
    A = np.sum(np.cos(wrun*tprime)**2,axis=1)**-0.5
    B = np.sum(np.sin(wrun*tprime)**2,axis=1)**-0.5
    
    A = A[:,np.newaxis]
    B = B[:,np.newaxis]

    # Note apparent typo in Scargle (1989), which has a plus 
    # sign (+) instead of a minus sign below. This only makes a 
    # difference in the periodogram if the input values in x are complex.

    ftx[k] = Fo*np.sum(A*x*np.cos(wrun*tprime) - i*B*x*np.sin(wrun*tprime),axis=1)
    
    # zero frequency 
    if k0.size != 0:
        ftx[k0] = np.sum(x)/np.sqrt(N)

    px = np.abs(ftx)**2/N
    
    return f, ftx, px

In [23]:
from scipy.signal import get_window

In [24]:
def lombscargle(t,x,ofac=4,hifac=1,t0=None,return_onesided=True,return_zero=False, window='boxcar',scaling='classical'):
    '''
    Compute the discrete Fourier transform and periodogram for unevenly-spaced 
    data using the Lomb-Scargle periodogram. Follows methods outlined in Scargle
    (1989). 
    
    INPUTS
    
    t - array of numerical time values (length N)
    x - array of data values, may be complex (length N)
    
    RETURNS
    
    f - array of frequencies
    ftx - array of complex coefficients
          discrete Fourier transform of x
    px - periodogram of ftx, proportional to |ftx|**2
         with the default "classical" scaling used in Scargle (1989), 
         px = (1/N)*|ftx|**2
    
    OPTIONAL PARAMETERS
    
    ofac - oversampling parameter 
           ratio of number of frequencies used to number of samples in x
           (default 4)        
    hifac - high frequency parameter
            ratio of highest frequency to pseudo-Nyquist frequency
            (default 1)  
    t0 - time origin
         reference point for phase calculation
         (default - None, first value in t array is used)
    return_onesided - boolean for returning a one-sided spectrum
        If True, return a one-sided spectrum for real data. 
        If False return a two-sided spectrum for real data.  
        Note that for complex data, a two-sided spectrum is always returned.
        (default - True)
    return_zero - boolean for evaluating zero frequency
        If True, include zero frequency. If False, do not include zero frequency.
        Uses expressions for the limit as frequency approaches zero, following
        Scargle (1989). 
        (default - False)
    window - String specifying desired window to use. See `scipy.signal.get_window` for 
        a list of windows and required parameters.
    scaling - Selects between computing the classical periodogram used by Scargle 
        ('classical') or the power spectral density ('density'). The classical 
        periodogram  has units of x**2. The power  spectral density has units of x**2/f. 
        The scaling determines how the periodogram px is calculated from the discrete 
        Fourier transform ftx:
        'classical': px = (1/N)*|ftx|**2, where N is the number of samples
        'density': px = (deltat/N)*|ftx|**2, where deltat is the average time step
      
    REFERENCE
    
    Scargle, J.D. (1989) Studies in astronomical time series analysis III: Fourier transforms,
    autocorrelation functions, and cross-correlation functions of unevenly spaced data. The
    Astrophysical Journal, 343, 874-887
    '''

    i = 1j # square root of -1
    N = len(x) # number of samples    
    
    wts = get_window(window,N)
    wts = N*wts/np.sum(wts) # make sum of weights equal to N
    
    x = x*wts  # apply window

    intm = np.mean(np.diff(t))

    flo = ((intm)**-1)/(len(x)*ofac)  # lowest freq
    fhi = hifac*(2*intm)**-1          # highest freq

    f = np.arange(flo,fhi+flo,flo)
    
    if return_zero == True:
        f = np.append(0,f)
        
    # if complex, evaluate two-sided spectrum regardless of user choice
    if np.any(np.iscomplex(x)):
        return_onesided = False
    
    # two-sided spectrum
    if return_onesided == False:
        if return_zero == True:
            f = np.append(-f[1:][::-1],f)
        else:
            f = np.append(-f[::-1],f)

    # time origin (reference point for phase calculation)
    if t0 is None:
        t0 = t[0] 

    # initialize DFT as array of complex numbers
    ftx = np.nan*np.ones(len(f)) + i*np.nan*np.ones(len(f))

    for k,fk in enumerate(f):
        wrun = 2*np.pi*fk # angular frequency    
        
        if fk == 0:
            # use well-defined limit as frequency approaches zero
            tau = np.sum(t)/N
            ftx[k] = np.sum(x)/np.sqrt(N)
            
        else:
            Fo = ((N/2)**0.5)*np.exp(-i*wrun*t0) 

            tau = np.arctan2(np.sum(np.sin(2*wrun*t)),np.sum(np.cos(2*wrun*t)))/(2*wrun)
            tprime = t - tau

            A = np.sum(np.cos(wrun*tprime)**2)**-0.5
            B = np.sum(np.sin(wrun*tprime)**2)**-0.5

            # Note apparent typo in Scargle (1989), which has a plus 
            # sign (+) instead of a minus sign below. This only makes a 
            # difference in the periodogram if the input values in x are complex.
            
            ftx[k] = Fo*np.sum(A*x*np.cos(wrun*tprime) - i*B*x*np.sin(wrun*tprime))

    if scaling == 'classical':
        px = np.abs(ftx)**2/N
    elif scaling == 'density':
        px = np.abs(ftx)**2/N*intm
    else:
        raise ValueError('Scaling argument not understood. Acceptable options are classical or density')
    
    return f, ftx, px

In [25]:
from tseries import lombscargle

usub = np.array(dfsub['u'])
vsub = np.array(dfsub['v'])

#rotary spectrum
f,ftx,Sx = lombscargle(tdsub,usub+1j*vsub,ofac=1,window='hanning')
icw, = np.where(f<0)
iccw, = np.where(f>0)
fcw = -f[icw]
Scw = Sx[icw]
fccw = f[iccw]
Sccw = Sx[iccw]

In [26]:
plt.figure()
plt.loglog(fcw,Scw)
plt.loglog(fccw,Sccw,'--')
plt.legend(['clockwise','counter-clockwise'])
plt.xlabel('frequency [cpd]')
plt.ylabel('[m$^2$ s$^{-2}$ cpd$^{-1}$]')
plt.title('rotary spectra - periodogram [Rover]')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'rotary spectra - periodogram [Rover]')

### Compare Lomb-Scargle with standard periodogram

In [27]:
fu,Su = periodogram(u,fs=24.,window='boxcar',scaling='density')
fu2,ftu2,Su2 = lombscargle(tdsub,usub,ofac=1,window='hanning',scaling='density')

In [28]:
intm = np.mean(np.diff(tdsub))
T = intm*len(tdsub)

In [29]:
plt.figure()
plt.loglog(fu,Su)
plt.loglog(fu2,Su2,'--')

FigureCanvasNbAgg()

In [30]:
i = 1j
fr,Sr = periodogram(u+i*v,fs=24.,window='hanning',scaling='density')
icw, = np.where(fr<0)
iccw, = np.where(fr>0)
fcw = -fr[icw]
Scw = Sr[icw]
fccw = fr[iccw]
Sccw = Sr[iccw]

fr2,ftr2,Sr2 = lombscargle(tdsub,usub+i*vsub,ofac=1,window='hanning',scaling='density')
icw2, = np.where(fr2<0)
iccw2, = np.where(fr2>0)
fcw2 = -fr2[icw2]
Scw2 = Sr2[icw2]
fccw2 = fr2[iccw2]
Sccw2 = Sr2[iccw2]

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/spectral.py:1623: UserWarning: Input data is complex, switching to return_onesided=False
  warnings.warn('Input data is complex, switching to '


In [31]:
plt.figure()
plt.loglog(fcw,Scw)
plt.loglog(fcw2,Scw2,'--')
plt.title('comparison of clockwise components')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'comparison of clockwise components')

In [32]:
plt.figure()
plt.loglog(fccw,Sccw)
plt.loglog(fccw2,Sccw2,'--')
plt.title('comparison of counter-clockwise components')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'comparison of counter-clockwise components')

### Lomb-Scargle periodogram for entire data set

In [33]:
#separating out the good data from the year 2016-2018

dfg = df.loc[(df.index != np.datetime64('NaT')) & 
             (df.index >= np.datetime64('2014-09-01 00:00:00')) &
             (df.index <= np.datetime64('2018-12-31 00:00:00')) &
             (np.isfinite(df['u']))]

tdg = np.array(dfg.index-dfg.index[0])/np.timedelta64(1,'D')

In [34]:
ug = np.array(dfg['u'])
vg = np.array(dfg['v'])

#rotary spectrum
fr,ftxr,Sxr = lombscargle(tdg,ug+1j*vg,ofac=1,window='hanning')
icw, = np.where(fr<0)
iccw, = np.where(fr>0)
fcw = -fr[icw]
Scw = Sxr[icw]
fccw = fr[iccw]
Sccw = Sxr[iccw]

In [35]:
plt.figure()
plt.loglog(fcw,Scw)
plt.loglog(fccw,Sccw,'--')
plt.legend(['clockwise','counter-clockwise'])
plt.xlabel('frequency [cpd]')
plt.ylabel('[m$^2$ s$^{-2}$ cpd$^{-1}$]')
plt.title('rotary spectra - periodogram [Rover]')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'rotary spectra - periodogram [Rover]')

In [36]:
#rotary spectrum
f,ftu,Su = lombscargle(tdg,ug,ofac=1,window='hanning')
f,ftv,Sv = lombscargle(tdg,vg,ofac=1,window='hanning')

In [37]:
plt.figure()
plt.loglog(f,Su)
plt.loglog(f,Sv,'--')
plt.legend(['u','v'])
plt.xlabel('frequency [cpd]')
plt.ylabel('[m$^2$ s$^{-2}$ cpd$^{-1}$]')
plt.title('component spectra - periodogram [Rover]')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'component spectra - periodogram [Rover]')

In [38]:
# average frequency bands
navg = 2 # number of frequency bands to average
nfreq = int(np.floor(len(f)/navg))  # number of freqencies in averaged spectrum

fwt = 1/navg
fm = np.zeros(nfreq)
Sum = np.zeros(nfreq)
Svm = np.zeros(nfreq)
for k in range(navg):
    fm = fm + fwt*f[1:(nfreq*navg)][::navg]
    Sum = Sum + fwt*Su[1:(nfreq*navg)][::navg]
    Svm = Svm + fwt*Sv[1:(nfreq*navg)][::navg]

In [39]:
plt.figure()
plt.loglog(fm,Sum)
plt.loglog(fm,Svm)
#plt.loglog(f,Sv,'--')
plt.legend(['u','v'])
plt.xlabel('frequency [cpd]')
plt.ylabel('[m$^2$ s$^{-2}$ cpd$^{-1}$]')
plt.title('component spectra - periodogram [Rover]')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'component spectra - periodogram [Rover]')